In [4]:
!pip install selenium


     ---------------------------------------- 6.9/6.9 MB 8.9 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import PIL
from PIL import Image
import io
import requests
import hashlib

In [33]:


# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r"C:\Users\donut\Downloads\chromedriver_win32\chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\donut\AppData\Local\Temp\ipykernel_41012\135492996.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [9]:
wd.get('https://google.com')


In [44]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR,"img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements(By.CSS_SELECTOR,'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            load_more_button = wd.find_element(By.CSS_SELECTOR,".mye4qd")
            if load_more_button:
             wd.execute_script("document.querySelector('.mye4qd').click();")
            return
        

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [23]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [45]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [68]:
target_path=r"C:\Users\donut\Desktop\imagedataset\vada pav"
search_term="vada pav"
search_and_download(search_term=search_term,driver_path=DRIVER_PATH,target_path=target_path,number_images=150)

C:\Users\donut\AppData\Local\Temp\ipykernel_41012\4098395253.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:


Found: 100 search results. Extracting links from 0:100
Found: 150 image links, done!
SUCCESS - saved https://i0.wp.com/thetwincookingproject.net/wp-content/uploads/2020/01/Vada-Pav-with-Dry-Garlic-Chutney-12-scaled.jpg?fit=1707%2C2560&ssl=1 - as C:\Users\donut\Desktop\imagedataset\vada pav\vada_pav\f785011953.jpg
SUCCESS - saved https://recipesofhome.com/wp-content/uploads/2022/09/vada-pav-recipe.jpg - as C:\Users\donut\Desktop\imagedataset\vada pav\vada_pav\2117ee73ff.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEojcr8vHkRwvWOSQ51FvpFudsqqUH0lW7Rg&usqp=CAU - as C:\Users\donut\Desktop\imagedataset\vada pav\vada_pav\f845630440.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfz8O-ON9DbI_uXh7NLXqnGmshN7qGfpmxRg&usqp=CAU - as C:\Users\donut\Desktop\imagedataset\vada pav\vada_pav\c719d58812.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSe9w-WmpuqfncrOCqqcDLSi8mRhHpqCD7hfA&usqp=CAU - as C:\Users\donut\Deskto